<a href="https://colab.research.google.com/github/kusuma2506/AI-AGENT/blob/main/health__care.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [3]:
df = pd.read_csv("health care.csv")

In [4]:
df = df.drop(columns=["Name", "Date of Admission", "Discharge Date", "Doctor", "Hospital"])
X = df.drop("Test Results", axis=1)
y = df["Test Results"]

In [7]:
categorical_cols = X.select_dtypes(include="object").columns.tolist()
numerical_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

In [9]:
encoders = {col: LabelEncoder().fit(X[col]) for col in categorical_cols}
for col in categorical_cols:
    X[col] = encoders[col].transform(X[col])

In [10]:
scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

In [11]:
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [13]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [14]:
from sklearn.metrics import accuracy_score, classification_report

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=target_encoder.classes_)

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
comparison = pd.DataFrame({
    'Actual': target_encoder.inverse_transform(y_test),
    'Predicted': target_encoder.inverse_transform(y_pred)
})

In [16]:
print(comparison.head(10))

         Actual Predicted
0        Normal    Normal
1      Abnormal  Abnormal
2  Inconclusive  Abnormal
3  Inconclusive    Normal


In [17]:
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 50.00%


In [18]:
import pickle

In [19]:
with open("healthcare_model.pkl", "wb") as f:
    pickle.dump({
        "model": model,
        "features": X.columns.tolist(),
        "encoders": encoders,
        "scaler": scaler,
        "target_encoder": target_encoder,
        "numerical_cols": numerical_cols,
        "categorical_cols": categorical_cols
    }, f)

In [20]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.2 MB/s eta 0:00:00


In [21]:
def predict(age, gender, blood_type, condition, insurance, bill, room, adm_type, medication):
    row = pd.DataFrame({
        "Age": [age],
        "Gender": [encoders["Gender"].transform([gender])[0]],
        "Blood Type": [encoders["Blood Type"].transform([blood_type])[0]],
        "Medical Condition": [encoders["Medical Condition"].transform([condition])[0]],
        "Insurance Provider": [encoders["Insurance Provider"].transform([insurance])[0]],
        "Billing Amount": [bill],
        "Room Number": [room],
        "Admission Type": [encoders["Admission Type"].transform([adm_type])[0]],
        "Medication": [encoders["Medication"].transform([medication])[0]]
    })
    row[numerical_cols] = scaler.transform(row[numerical_cols])
    result = model.predict(row)[0]
    return target_encoder.inverse_transform([result])[0]

In [22]:
import gradio as gr

In [27]:
gr.Interface(
    fn=predict,
    inputs=[
        gr.Number(label="Age"),
        gr.Dropdown(choices=encoders["Gender"].classes_.tolist(), label="Gender"),
        gr.Dropdown(choices=encoders["Blood type"].classes_.tolist(), label="Blood type"),
        gr.Dropdown(choices=encoders["Medical Condition"].classes_.tolist(), label="Medical Condition"),
        gr.Dropdown(choices=encoders["Insurance Provider"].classes_.tolist(), label="Insurance Provider"),
        gr.Number(label="Billing Amount"),
        gr.Number(label="Room Number"),
        gr.Dropdown(choices=encoders["Admission Type"].classes_.tolist(), label="Admission Type"),
        gr.Dropdown(choices=encoders["Medication"].classes_.tolist(), label="Medication")
    ],
    outputs=gr.Text(label="Predicted Test Result"),
    title="Healthcare Test Result Predictor"
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fdc76d828e5ad8ce90.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [26]:
df

,Age,Gender,Blood type,Medical Condition,Insurance Provider,Billing Amount,Room Number,Admission Type,Medication,Test Results
0,30,Male,B-,Cancer,Blue Cross,18856.28131,328,Urgent,Paracetamol,Normal
1,62,Male,A+,Obesity,Medicare,33643.32729,265,Emergency,Ibuprofen,Inconclusive
2,76,Female,A-,Obesity,Aetna,27955.09608,205,Emergency,Aspirin,Normal
3,28,Female,O+,Diabetes,Medicare,37909.78241,450,Elective,Ibuprofen,Abnormal
4,43,Female,AB+,Cancer,Aetna,14238.31781,458,Urgent,Penicillin,Abnormal
5,36,Male,A+,Asthma,UnitedHealthcare,48145.11095,389,Urgent,Ibuprofen,Normal
6,21,Female,AB-,Diabetes,Medicare,19580.87234,389,Emergency,Paracetamol,Inconclusive
7,20,Female,A+,Cancer,Cigna,45820.46272,277,Emergency,Paracetamol,Inconclusive
8,82,Male,AB+,Asthma,Cigna,50119.22279,316,Elective,Aspirin,Abnormal
9,58,Female,AB-,Cancer,UnitedHealthcare,19784.63106,249,Elective,Paracetamol,Inconclusive
